# Extract GitHub contributions over time

Uses [GitPython](http://gitpython.readthedocs.io/en/stable/).

In [4]:
import collections
import pathlib
import re
import datetime
import git
import pandas
import pytz

In [5]:
repo = git.Repo(path = '../')
repo.head.reference = "master"
print(repo.head.commit)

89adbc3fed98ebf280331011aef5bf804bcbf77c


In [6]:
# These patterns are reversed because our diffs are backwards
# -R (R=True, suggested in https://git.io/vdigv) doesn't seem to work
added_pattern = re.compile(r'^-(.+)', flags=re.MULTILINE)
deleted_pattern = re.compile(r'^\+(.+)', flags=re.MULTILINE)
pattern_alphanum = re.compile(r'\w')

def get_word_count(text):
    """
    Compute word count of text. Splits by whitespace and only retains
    words with at least one alphanumeric character [a-zA-Z0-9_].
    """
    words = text.split()
    words = list(filter(pattern_alphanum.match, words))
    return len(words)

def get_commit_stats(commit):
    """
    Return addition and deletion (based on `git diff --word-diff`) for a commit.
    
    https://git-scm.com/docs/git-diff
    http://gitpython.readthedocs.io/en/stable/reference.html#module-git.diff
    """
    diffs = commit.diff(
        other=commit.parents[0] if commit.parents else git.NULL_TREE,
        paths=['sections/*.md', 'content/*.md'],
        create_patch=True,
        word_diff='porcelain',
    )

    delta = collections.OrderedDict()
    for key in 'words_added', 'words_deleted', 'characters_added', 'characters_deleted':
        delta[key] = 0
    for diff in diffs:
        diff_str = str(diff)
        additions = added_pattern.findall(diff_str)
        deletions = deleted_pattern.findall(diff_str)

        # Attempt to ignore relocated lines
        additions, deletions = (
            [x for x in additions if get_word_count(x) < 4 or x not in set(deletions)],
            [x for x in deletions if get_word_count(x) < 4 or x not in set(additions)],
        )
        for added in additions:
            delta['words_added'] += get_word_count(added)
            delta['characters_added'] += len(added)
        for deleted in deletions:
            delta['words_deleted'] += get_word_count(deleted)
            delta['characters_deleted'] += len(deleted)
    return(delta)

In [7]:
rows = list()
# We want to filter out anything from the Manubot development project that predates the COVID-19 project
# The COVID-19 project began on 3/20/2020, but let's build in one extra day just in case timezones cause issues
tz = pytz.timezone('America/New_York')
project_start = datetime.datetime(2020, 3, 19, 0, 0, 0, tzinfo=tz)
for commit in repo.iter_commits():
    if commit.authored_datetime < project_start:
        continue
    row = collections.OrderedDict()
    row['commit'] = commit.hexsha
    row['author_name'] = commit.author.name
    row['author_email'] = commit.author.email
    row['committer_name'] = commit.committer.name
    row['authored_datetime'] = commit.authored_datetime
    row['committed_datetime'] = commit.committed_datetime
    row['summary'] = commit.summary
    row['count'] = commit.count()
    row['merge'] = int(len(commit.parents) > 1)
    row['parents'] = ', '.join(x.hexsha for x in commit.parents)
    row.update(get_commit_stats(commit))
    rows.append(row)

rows = list(reversed(rows))
commit_df = pandas.DataFrame(rows)
backup_times = commit_df["authored_datetime"]

In [8]:
commit_df.tail(5)

,commit,author_name,author_email,committer_name,authored_datetime,committed_datetime,summary,count,merge,parents,words_added,words_deleted,characters_added,characters_deleted
4997,b9ca24fc1dc5c14640912224f86105dfe5a0b4cc,Anthony Gitter,agitter@users.noreply.github.com,GitHub,2021-09-09 16:33:09-05:00,2021-09-09 16:33:09-05:00,Update JSON syntax,5293,0,32663258d3537df26bc6d06d81db8c4d37f9e733,0,0,0,0
4998,0fcecefe7120abf28a6e20f8e6ab7e0382b908e3,HM Rando,halie.rando@cuanschutz.edu,GitHub,2021-09-09 18:03:03-04:00,2021-09-09 18:03:03-04:00,Apply suggestions from code review,5294,0,b9ca24fc1dc5c14640912224f86105dfe5a0b4cc,35,28,255,171
4999,78418ae1e8e3f4c84f2d97310745ffd8d61d02d7,HM Rando,halie.rando@pennmedicine.upenn.edu,HM Rando,2021-09-09 18:05:09-04:00,2021-09-09 18:05:09-04:00,dictionary updates,5295,0,0fcecefe7120abf28a6e20f8e6ab7e0382b908e3,0,0,0,0
5000,48203f74d8283154d866d14aef2c7e87b337cd67,HM Rando,halie.rando@cuanschutz.edu,GitHub,2021-09-09 18:05:28-04:00,2021-09-09 18:05:28-04:00,Merge branch 'master' into jf-edits,5316,1,"78418ae1e8e3f4c84f2d97310745ffd8d61d02d7, 2bcf...",529,0,3952,93
5001,89adbc3fed98ebf280331011aef5bf804bcbf77c,Anthony Gitter,agitter@users.noreply.github.com,GitHub,2021-09-09 21:31:23-05:00,2021-09-09 21:31:23-05:00,Merge pull request #1025 from rando2/jf-edits,5317,1,"2bcf287241085c340b0346af1b798cd9c84ac113, 4820...",1092,539,8315,4086


## Manual author name fixes

In [9]:
# Update any users whose GitHub names don't match their authorship names
# Use information specified in tab-delim file renamer.tsv

path = pathlib.Path('renamer.tsv')
renamer = dict()
if path.exists():
    df = pandas.read_csv(path, sep='\t')
    renamer = dict(zip(df.name, df.rename_to))
len(renamer)

35

In [10]:
for column in 'author_name', 'committer_name':
    print(f"Unique {column}s before renaming {commit_df[column].nunique()}")
    commit_df[column].replace(renamer, inplace=True)
    print(f"Unique {column}s after renaming {commit_df[column].nunique()}")

Unique author_names before renaming 62
Unique author_names after renaming 52
Unique committer_names before renaming 20
Unique committer_names after renaming 19


In [11]:
commit_df.head(2)

,commit,author_name,author_email,committer_name,authored_datetime,committed_datetime,summary,count,merge,parents,words_added,words_deleted,characters_added,characters_deleted
0,3f8cdd531916e7cfb3a295f53142c6930ad5c124,Halie Rando,halie.rando@pennmedicine.upenn.edu,Halie Rando,2020-03-20 15:18:05-04:00,2020-03-20 15:18:05-04:00,initial commit,316,0,1780fac0ac6bba1260a9da3886061730fa5d2765,146,0,1000,0
1,02b07f0f32bdcd7075dde7370bb08e6e36d908a1,Halie Rando,halie.rando@pennmedicine.upenn.edu,Halie Rando,2020-03-20 18:11:40-04:00,2020-03-20 18:11:40-04:00,updated readme and abstract,317,0,3f8cdd531916e7cfb3a295f53142c6930ad5c124,85,61,630,414


In [12]:
# Fix date format to be compatible with R
utc_dt = list()
for index, value in commit_df["authored_datetime"].items():
    utc_dt.append(value.astimezone(pytz.utc))
commit_df["authored_datetime"] = pandas.Series(utc_dt)
commit_df["authored_datetime"] = commit_df["authored_datetime"].dt.date

## Export

In [13]:
commit_df.to_csv('commits.tsv', sep='\t', index=False)

In [14]:
# Check and see whether these need to be cleaned up!
print(commit_df["author_name"].unique())

['Halie Rando' 'Casey Greene' 'Anthony Gitter' 'Michael Robson'
 'Ronnie Russell' 'Simina Boca' 'Nils Wellhausen' 'Christian Brueffer'
 'Ronan Lordan' "Lucy D'Agostino McGowan" 'Sandipan Ray'
 'Anna Ada Dattoli' 'Ryan Hagenson' 'Jeff Field' 'Adam MacLean'
 'Ryan Velazquez' 'Fengling Hu' 'John P Barton' 'Bharath Ramsundar'
 'Alexandra Lee' 'Diane Rafizadeh' 'John J. Dziak' 'Nafisa Jadavji'
 'Elizabeth Sell' 'Marouen Ben Guebila' 'Jinhui Wang' 'David Mai'
 'Yoson Park' 'Likhitha Kolla' 'Ashwin Skelly' 'Daniel Himmelstein'
 'J. Brian Byrd' 'Soumita Ghosh' 'David Manheim' 'Matthias Fax'
 'Yael Marshall' 'Vikas Bansal' 'Vincent Rubinetti' 'Sebastian Karcher'
 'Stephen Capone' 'Yanjun Qi' 'Yuchen Sun' 'Lamonica Shinholster'
 'Sergey Knyazev' 'Dimitri Perrin' 'Greg Szeto' 'C. Titus Brown'
 'Temitayo Lukan' 'Tiago Lubiana' 'Nick Fry' 'Amruta Naik' 'Yusha Sun']


In [15]:
# Go back to the external-resources branch
repo.head.reference = "contrib-viz"